In [2]:
import pandas as pd
semanticforce_file = open('kyiv_digital_sentiment_annotation - Annotator X.tsv')
data = pd.read_csv(semanticforce_file,sep='\t')

In [3]:
with open('emotions_sentiment_youscan - Annotator 1 - Veronika.tsv','r') as file3:
    data3=list(file3)
    data3=[e.replace('\n','').split('\t')[0:2] for e in data3[1:652]]
data2 = pd.DataFrame(data3, columns=['content', 'label'])
df = pd.concat([data, data2], ignore_index=True)
label_mapping = {'Neutral': 0, 'Negative': 1, 'Positive': 2,'Very Positive':2,'Very Negative':1, 'Mixed':1}
df['label'] =df['label'].map(label_mapping)
df = df.dropna()
df['label'] =df['label'].astype(int)

In [4]:
import pandas as pd
from datasets import Dataset, load_metric
from sklearn.metrics import roc_auc_score, f1_score, matthews_corrcoef
from sklearn.model_selection import train_test_split

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [5]:
train_df, val_df = train_test_split(df, test_size=0.1, stratify=df['label'], random_state=42)

# Convert DataFrame to Dataset
train_dataset = Dataset.from_pandas(train_df)
val_dataset = Dataset.from_pandas(val_df)

In [6]:
val_dataset

Dataset({
    features: ['content', 'label', '__index_level_0__'],
    num_rows: 122
})

In [29]:
len(preds),len(val_dataset['label'])

(122, 122)

Generating train split: 100%|█| 128549/128549 [00:00<00:00, 14305
Generating test split: 100%|█| 52294/52294 [00:00<00:00, 2380655.


In [36]:
len(dataset)

2

In [40]:
len(dataset['train']['text'])

128549

In [41]:
dataset['train']['text'][0]

"чому редагування, зроблені за моїм ім'ям жорстких мета був повернений вони були не вандалізм щойно закрився на якомусь газі після того як я голосував на нових ляльках fac і будьласка не вилучіть шаблон зі сторінки промови з тих пір, як миттєвий відставку зараз"

In [42]:
len(dataset['test']['text'])

52294

In [43]:
dataset['test']['text'][0:10]

['Дякую за розуміння, я думаю, дуже високо про вас і не повернеться без обговорення',
 'Боже мій, це місце жахливе.',
 'хтось завжди буде намагатися додати релігію дійсно Ви маєте на увазі те, як люди незмінно додавали релігію до samuel backetbox і чому ви турбуєтеся, щоб згадати довгосмертних повністю не існує вплив ви просто ви робите висмоктування лайно на льоту для порівняння єдине явне визнання в всій статті амосів, що він особисто є еврей в категорії',
 "каже, що це тип інституції потрібний в цьому випадку, тому що є три рівні сонячних університетських центрів та докторського грантування університетів штату Університети, це необхідно в цьому випадку, щоб прояснити, що ub є сонячним центром він каже, що навіть в університеті Бінгемтоні в університеті штату Нью-Йорк і кам'яний тюрк, що намагається сказати не тому, що я повністю правий в цьому випадку",
 'Перш ніж додавати новий продукт до списку, переконайтеся, що його значення додається перед додаванням нового продукту до списку, 

In [44]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer

# Load the model and tokenizer
model = AutoModelForSequenceClassification.from_pretrained('./fine_tuned_model2')
tokenizer = AutoTokenizer.from_pretrained('./fine_tuned_model2')

# Set the model to evaluation mode
model.eval()

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(29564, 312, padding_idx=0)
      (position_embeddings): Embedding(512, 312)
      (token_type_embeddings): Embedding(2, 312)
      (LayerNorm): LayerNorm((312,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-2): 3 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=312, out_features=312, bias=True)
              (key): Linear(in_features=312, out_features=312, bias=True)
              (value): Linear(in_features=312, out_features=312, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=312, out_features=312, bias=True)
              (LayerNorm): LayerNorm((312,), eps=1e-12, e

In [45]:
text = "Your input text here"

# Tokenize the input text
inputs = tokenizer(text, return_tensors='pt')

# Make predictions
with torch.no_grad():
    outputs = model(**inputs)
    logits = outputs.logits
    predicted_class = torch.argmax(logits, dim=1).item()

print(f'Predicted class: {predicted_class}')

Predicted class: 0


In [ ]:
 #{'Neutral': 0, 'Negative': 1, 'Positive': 2,'Very Positive':2,'Very Negative':1, 'Mixed':1}
